In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# Class 12: Introduction to the `linearsolve` Module

In general, dynamic stochastic general equilibrium (DSGE) models do not admit analytic (i.e., pencil-and-paper) solutions and they are time-consuming to work with. The `linearsolve` module approximates, solves, and simulates DSGE models and therefore makes DSGE models easier to use.

## Installing `linearsolve`

`linearsolve` is not included in the Anaconda Python installation and so before you can import it, you need to download and install the `linearsolve` package from PyPI, the Python Package Index. To install, run the following command in a Jupyter Notebook cell:

    pip install linearsolve
    
Alternatively, you can install from the command line. In Windows, open the Anaconda Prompt and in Mac, open the Terminal and run the following commmand. You only have to install the package once.

## More information about `linearsolve`

Documentation, including examples, for `linearsolve` is availabe here:

https://www.briancjenkins.com/linearsolve/docs/build/html/index.html

You can view the code in this GitHub repository: https://github.com/letsgoexploring/linearsolve

In [ ]:
# Import the linearsolve under the 'ls' namespace


## Example: A One-Equation Model of TFP

Consider the following AR(1) specification for $\log$ TFP:

\begin{align}
\log A_{t+1} & = \rho \log A_t + \epsilon_{t+1},
\end{align}

where $\epsilon_{t+1} \sim \mathcal{N}(0,\sigma^2)$. The goal is to simulate 

\begin{align}
\log (A_{t}/\bar{A}) & \approx \frac{A_t - \bar{A}}{\bar{A}},
\end{align}

where $\bar{A}$ is the nonstochastic (i.e., $\epsilon_t = 0$) steady state value of $A_t$. Let's compute the model simulation with `linearsolve`. To do this we need to do several things:

1. Create a Pandas series that stores the names of the parameters of the model.
2. Define a function that returns the equilibrium conditions of the model solved for zero.
3. Initialize an instance of the `linearsolve.model` class
4. Compute and input the steady state of the model.
5. Approximate and solve the model.
6. Compute simulations of the model.

Use the following values for the simulation:

| $$\rho$$ | $$\sigma$$ |
|----------|------------|
| 0.95     | 0.01       |

**Step 1:** Create a variable called `parameters` that stores parameter values as a Pandas Series.

In [ ]:
# Create a variable called 'parameters' that stores the model parameter values in a Pandas Series


# Print stored parameter values


In [ ]:
# Create a variable called 'sigma' that stores the value of sigma


In [ ]:
# Create variable called 'var_names' that stores the variable names in a list with exogenous state variables
# ordered first, endogenous state variables ordered second, and control variables ordered last.


# Create variable called 'shock_names' that stores an exogenous shock name for each exogenous state variable.


**Step 2:** Define a function called `equilibrium_equations` that evaluates the equilibrium 
equations of model solved for zero. The function should accept three arguments:

1. `variables_forward`: Values of $t+1$-dated variables
2. `variables_current`: Values of $t$-dated variables
3. `parameters`: Pandas Series with the parameters for the model

The function should return a NumPy array of the model's equilibrium conditions solved for zero.

In [ ]:
# Define a function that evaluates the equilibrium conditions of the model solved for zero. PROVIDED
def equilibrium_equations(variables_forward,variables_current,parameters):

    # Create a variable called 'p' that storres the model's parameters. PROVIDED.
    p = parameters

    # Create variable called 'cur' that stores the values of current (date t) variables. PROVIDED.
    cur = variables_current

    # Create variable called 'fwd' that stores the values of one-period-ahead (date t+1) variables. PROVIDED.
    fwd = variables_forward

    # Create variable called 'tfp_proc' that returns the law of motion for TFP solved for zero (exogenous shock excluded)


    # Return equilibrium conditions stacked in a numpy array. PROVIDED.
    return np.array([



**Step 3:** Initialize a model instance using the `ls.model()` function. The 

1. `equations`: Name of function that stores the equilibrium conditions
2. `n_states`: Number of *state* variables (i.e., variables that are *predetermined*)
3. `n_exo_states`: Number of *exogenous* state variables
4. `var_names`: List of the names of the endogenous variables
5. `shock_names`: List of the names of the exogenous shocks
6. `parameters`: Pandas Series of parameter values

In [ ]:
# Initialize the model into a variable named 'ar1_model' using the ls.model() function




**Step 4:** Set the steady state of the model. Either use the `.compute_ss()` method which requires an initial guess of what the steady state is. Or set the steady state `.ss` attribute directly.

In [ ]:
# Compute the steady state numerically using .compute_ss() method of ar1_model



In [ ]:
# Set the .ss attribute of ar1_model directly


**Step 5:** Compute a transform the model into a log-linear approximation around the nonstochastic steady state. Then rewrite the equilibrium conditions so that all endogenous variables are expressed as linear functions of state variables and exogenous shocks.

In [ ]:
# Appproximate and solve using the .approximate_and_solve() method of ar1_model


**Step 6:** Simulate the model using one of the following methods:

1. `.impulse()`: Compute impulse responses to a one-time shock to each exogenous variables. Results stored in `.irs` attribute.
2. `.stoch_sim()`: Compute stochastic simulation. Results stored in `.simulated` attribute.

First, we'll compute an impulse response simulation. Let's consider the effect of a one time shock to $\epsilon$ of 0.01 in period 5. Simulate 41 periods.

In [ ]:
# Compute impulse response of a to a one-time shock to e_a


The impulse response simulations are stored in the `.irs` attribute as a dictionary with keys equal to the names of the exogenous shocks. 

In [ ]:
# Print ar1_model.irs


Let's look at the first 10 rows of the `'e_a'` element of `ar1_model.irs`.

In [ ]:
# Print first 10 rows of the element in ar1_model.irs that corresponds to the shock to TFP


In [ ]:
# Plot simulated impulse response to e_a


Next, we'll use the `.stoch_sim()` method to compute a stochastic simulation. The method takes arguments:

1. `seed`: Seed of NumPy RNG. (Optional)
2. `T`: Number of periods to simulate
3. `covMat`: Covariance matrix for exogenous shock process

The simulation should be for 201 periods.

In [ ]:
# Compute stochastic simulation


# Print the first 10 rows of `model.simulated`


The stochastic simulations are stored in the `.stoch_sim` attribute as a Pandas `DataFrame`.

In [ ]:
# Plot the stochastic simulation


## Example 2: The Stochastic Solow Growth Model Revisited

Now consider the following system of equations:

\begin{align}
Y_t & = A_t K_t^{\alpha} \\
K_{t+1} & = sY_t + (1-\delta) K_t\\
\log A_{t+1} & = \rho \log A_t + \epsilon_{t+1}
\end{align}

where $\epsilon_{t+1} \sim \mathcal{N}(0,\sigma^2)$. Let's simulate the model with `linearsolve`. Before proceding, let's also go ahead and rewrite the model with all variables moved to the lefthand side of the equations:

\begin{align}
0 & = A_t K_t^{\alpha} - Y_t \\
0 & = sY_t + (1-\delta) K_t - K_{t+1}\\
0 & = \rho \log A_t + \epsilon_{t+1} - \log A_{t+1}
\end{align}

Capital and TFP are called *state variables* because they're $t+1$ values are predetermined. Output is called a *costate* or *control* variable. Note that the model as 3 endogenous variables with 2 state variables. 

Use the following values for the simulation:

| $$\rho$$ | $$\sigma$$ | $$s$$ | $$\alpha$$ | $$\delta $$ | $$T$$  |
|----------|------------|-------|------------|-------------|--------|
| 0.75     | 0.006      | 0.1   | 0.35       |  0.025      | 201    |

### Initialization, Approximation, and Solution

The next several cells initialize the model in `linearsolve` and then approximate and solve it.

In [ ]:
# Create a variable called 'parameters' that stores the model parameter values in a Pandas Series


# Print the model's parameters


In [ ]:
# Create a variable called 'sigma' that stores the value of sigma


In [ ]:
# Create variable called 'var_names' that stores the variable names in a list with exogenous state variables
# ordered first, endogenous state variables ordered second, and control variables ordered last.


# Create variable called 'shock_names' that stores an exogenous shock name for each exogenous state variable.


In [ ]:
# Define a function that evaluates the equilibrium conditions of the model solved for zero. PROVIDED
def equilibrium_equations(variables_forward,variables_current,parameters):

    # Parameters. PROVIDED
    p = parameters

    # Current variables. PROVIDED
    cur = variables_current

    # Forward variables. PROVIDED
    fwd = variables_forward

    # Production function


    # Capital evolution


    # Exogenous tfp


    # Stack equilibrium conditions into a numpy array




Next, initialize the model.

In [ ]:
# Initialize the model into a variable named 'solow_model'




In [ ]:
# Compute the steady state numerically using .compute_ss() method of solow_model


# Print the computed steady state


In [ ]:
# Find the log-linear approximation around the non-stochastic steady state and solve using .approximate_and_solve() method of solow_model


### A Few Details About the Approximation (Optional)

The previous step constructs a log-linear approximation of the model and then solves for the endogenous variables as functions of the state variables and exogenous shocks only.

View the approximated model by calling the `.approximated()` method.

In [ ]:
# Print the log-linear approximation to the models's equilibrium conditions


Each variable represents the log-deviation from the steady state of the respective variable in our model. For example, the variable `y[t]` means $\log(Y_t) - \log(Y)$ in terms of the stochastic Solow model. But how do these equations relate to the original model?

The first equation appears to be:

\begin{align}
0 &= -2.1095\cdot a_t-0.7383\cdot k_t + 2.1095\cdot y_t
\end{align}

Note that dividing by 2.1095 and solving for $y_t$ yields:

\begin{align}
y_t &= a_t +0.3499\cdot k_t,
\end{align}

so the coefficient on $k_t$ appears to be close to $\alpha=0.35$. We can derive this linear equation directly. First, start with the production function:

\begin{align}
Y_t &= A_t K_t^{\alpha}.
\end{align}

Then divide both sides by steady state output:

\begin{align}
\frac{Y_t}{Y} &= \frac{A_t K_t^{\alpha}}{AK^{\alpha}} \, = \, \frac{A_t}{A}\frac{K_t^{\alpha}}{K^{\alpha}}.
\end{align}

Then, take the log of both sides:
\begin{align}
\log\left(\frac{Y_t}{Y} \right)&= \log\left(\frac{A_t}{A}\right) + \alpha\log\left(\frac{K_t}{K}\right).
\end{align}

finally, letting $y_t = \log(Y_t/Y)$, k_t = $\log(K_t/K)$, and $a_t = \log(A_t/A)$, we have:

\begin{align}
y_t &= a_t + \alpha k_t.
\end{align}

However, understanding this process isn't as important as being able to interpret the graphs and statistics that compute using the output of `linearsolve`.

### A Few Details About the Solution (Optional)

It's also worth seeing what it means for a model to be *solved*. After `linearsolve` computes the log-linear approximation to the model, it solves for each endogenous variable as a function of state variables only. View the solved model by calling the `.solved()` method.

In [ ]:
# Print the solved model


### Impulse Responses

Compute a 41 period impulse responses of the model's variables to a 0.01 unit shock to TFP in period 5.

In [ ]:
# Compute impulse responses


# Print the first 10 rows of the computed impulse responses.


In [ ]:
# Plot the computed impulse responses to a TFP shock




### Stochastic Simulation

Compute a 201 period stochastic simulation of the model's variables. Set the variance of $\epsilon_t$ to $\sigma^2$ and the variance of the shock to capital to 0 so that the covariance matrix for the shock process is:

\begin{align}
\text{Covariance matrix} & = \left[\begin{array}{cc} \sigma^2 \end{array} \right]
\end{align}

In [ ]:
# Compute stochastic simulation and print the simulated values.


# Print first 10 rows of model.simulated


In [ ]:
# Plot the computed stochastic simulation




In [ ]:
# Compute standard deviations of simulated TFP, output, and capital


In [ ]:
# Compute correlation coefficients of simulated TFP, output, and capital
